<a href="https://colab.research.google.com/github/sanuthit/Risk-Based-Motor-Insurance-Premium-Calculation-System-/blob/Renewal-risk-model/Renewal_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import pandas as pd
import numpy as np

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
DATA_DIR = "/content/drive/MyDrive/Data/Datasets/raw"
POLICY_PATH = f"{DATA_DIR}/policy_master.csv"
CLAIMS_PATH = f"{DATA_DIR}/claims_dataset.csv"

policy_master = pd.read_csv(POLICY_PATH)
claims = pd.read_csv(CLAIMS_PATH)

print("policy_master:", policy_master.shape)
print("claims:", claims.shape)

display(policy_master.head())
display(claims.head())


policy_master: (60000, 5)
claims: (104195, 7)


,customer_id,policy_id,policy_start_date,policy_expiry_date,renewal_date
0,C000001,P000001,2023-09-09,2024-09-09,2024-09-09
1,C000002,P000002,2023-02-14,2024-02-14,2024-02-14
2,C000003,P000003,2023-09-12,2024-09-12,2024-09-12
3,C000004,P000004,2024-10-09,2025-10-09,2025-10-09
4,C000005,P000005,2024-02-29,2025-02-28,2025-02-28


,claim_id,policy_id,claim_date,claim_type,claim_amount,claim_status,is_major_claim
0,CL0000001,P000001,2024-06-12,Accident,71918,Paid,0
1,CL0000002,P000001,2024-07-29,Accident,441899,Paid,0
2,CL0000003,P000002,2023-10-12,Accident,503652,Paid,0
3,CL0000004,P000002,2023-02-16,Accident,1322535,Paid,1
4,CL0000005,P000003,2023-12-05,Accident,2118271,Rejected,1


In [91]:
# Dates
for c in ["policy_start_date","policy_expiry_date","renewal_date"]:
    policy_master[c] = pd.to_datetime(policy_master[c], errors="coerce")

claims["claim_date"] = pd.to_datetime(claims["claim_date"], errors="coerce")

# Amounts / flags
claims["claim_amount"] = pd.to_numeric(claims["claim_amount"], errors="coerce")
claims["is_major_claim"] = pd.to_numeric(claims["is_major_claim"], errors="coerce").fillna(0).astype(int)

# Basic cleanup
claims = claims.dropna(subset=["policy_id","claim_id","claim_date","claim_amount"])
policy_master = policy_master.dropna(subset=["policy_id","renewal_date"])

print("After cleanup -> policy_master:", policy_master.shape, "| claims:", claims.shape)

After cleanup -> policy_master: (60000, 5) | claims: (104195, 7)


In [92]:

# Validations (should be TRUE for your realistic v2 data)
print("Unique policies in master:", policy_master["policy_id"].is_unique)
print("Unique customers in master:", policy_master["customer_id"].is_unique)

# Every claim should map to a policy (left join check)
missing_pol = claims.loc[~claims["policy_id"].isin(policy_master["policy_id"]), "policy_id"].nunique()
print("Claims with missing policy_id in master:", missing_pol)


Unique policies in master: True
Unique customers in master: True
Claims with missing policy_id in master: 0


Merge claims with policy

In [93]:
df = claims.merge(
    policy_master[["policy_id","renewal_date","policy_start_date","policy_expiry_date"]],
    on="policy_id",
    how="left"
)

# Days before renewal (time distance)
df["days_before_renewal"] = (df["renewal_date"] - df["claim_date"]).dt.days

# Sanity check: should be >=0 and <= ~365 (claims are within the policy year)
print(df["days_before_renewal"].describe())
print("Negative days_before_renewal rows:", (df["days_before_renewal"] < 0).sum())
print("Claims after expiry rows:", (df["claim_date"] > df["policy_expiry_date"]).sum())
print("Claims before start rows:", (df["claim_date"] < df["policy_start_date"]).sum())

display(df.head())

count    104195.000000
mean        183.400931
std         105.478305
min           1.000000
25%          92.000000
50%         184.000000
75%         275.000000
max         366.000000
Name: days_before_renewal, dtype: float64
Negative days_before_renewal rows: 0
Claims after expiry rows: 0
Claims before start rows: 0


,claim_id,policy_id,claim_date,claim_type,claim_amount,claim_status,is_major_claim,renewal_date,policy_start_date,policy_expiry_date,days_before_renewal
0,CL0000001,P000001,2024-06-12,Accident,71918,Paid,0,2024-09-09,2023-09-09,2024-09-09,89
1,CL0000002,P000001,2024-07-29,Accident,441899,Paid,0,2024-09-09,2023-09-09,2024-09-09,42
2,CL0000003,P000002,2023-10-12,Accident,503652,Paid,0,2024-02-14,2023-02-14,2024-02-14,125
3,CL0000004,P000002,2023-02-16,Accident,1322535,Paid,1,2024-02-14,2023-02-14,2024-02-14,363
4,CL0000005,P000003,2023-12-05,Accident,2118271,Rejected,1,2024-09-12,2023-09-12,2024-09-12,282


Create time-window flags

In [94]:
df["in_last_1_year"]  = df["days_before_renewal"].between(0, 365).astype(int)
df["in_last_3_years"] = df["days_before_renewal"].between(0, 365*3).astype(int)

# Pending claims should usually be near expiry (optional sanity check)
if "Pending" in df["claim_status"].astype(str).unique():
    print("Pending claims days_before_renewal summary:")
    display(df.loc[df["claim_status"]=="Pending", "days_before_renewal"].describe())

Pending claims days_before_renewal summary:


,days_before_renewal
count,5092.000000
mean,30.421249
std,17.288949
min,1.000000
25%,15.000000
50%,30.000000
75%,45.000000
max,60.000000


Aggregate to policy-level features

In [95]:
claim_agg = df.groupby("policy_id").agg(
    total_claims=("claim_id","count"),
    total_claim_amount=("claim_amount","sum"),
    avg_claim_amount=("claim_amount","mean"),
    max_claim_amount=("claim_amount","max"),
    major_claim_count=("is_major_claim","sum"),
    paid_claims=("claim_status", lambda s: (s=="Paid").sum()),
    rejected_claims=("claim_status", lambda s: (s=="Rejected").sum()),
    pending_claims=("claim_status", lambda s: (s=="Pending").sum()),
    claims_last_1_year=("in_last_1_year","sum"),
    claims_last_3_years=("in_last_3_years","sum"),
    last_claim_date=("claim_date","max"),
).reset_index()

print("claim_agg:", claim_agg.shape)
display(claim_agg.head())

claim_agg: (52731, 12)


,policy_id,total_claims,total_claim_amount,avg_claim_amount,max_claim_amount,major_claim_count,paid_claims,rejected_claims,pending_claims,claims_last_1_year,claims_last_3_years,last_claim_date
0,P000001,2,513817,256908.5,441899,0,2,0,0,2,2,2024-07-29
1,P000002,2,1826187,913093.5,1322535,1,2,0,0,2,2,2023-10-12
2,P000003,2,4155749,2077874.5,2118271,2,1,1,0,2,2,2023-12-05
3,P000004,2,196571,98285.5,118691,0,2,0,0,2,2,2025-05-23
4,P000006,1,2031778,2031778.0,2031778,1,1,0,0,1,1,2023-02-16


Add 1-year severity features

In [96]:
last1 = df[df["in_last_1_year"]==1].groupby("policy_id").agg(
    total_claim_amount_last_1_year=("claim_amount","sum"),
    largest_claim_amount_last_1_year=("claim_amount","max"),
    had_major_claim_last_1_year=("is_major_claim","max"),
).reset_index()

print("last1:", last1.shape)
display(last1.head())


last1: (52712, 4)


,policy_id,total_claim_amount_last_1_year,largest_claim_amount_last_1_year,had_major_claim_last_1_year
0,P000001,513817,441899,0
1,P000002,1826187,1322535,1
2,P000003,4155749,2118271,1
3,P000004,196571,118691,0
4,P000006,2031778,2031778,1


In [97]:
# ✅ CRITICAL: start from policy_master so 0-claim (NCB) policies are included
policy_features = policy_master.merge(claim_agg, on="policy_id", how="left")
policy_features = policy_features.merge(last1, on="policy_id", how="left")

# Fill numeric claim fields with 0 for zero-claim policies
num_cols = [
    "total_claims","total_claim_amount","avg_claim_amount","max_claim_amount","major_claim_count",
    "paid_claims","rejected_claims","pending_claims",
    "claims_last_1_year","claims_last_3_years",
    "total_claim_amount_last_1_year","largest_claim_amount_last_1_year","had_major_claim_last_1_year"
]
for c in num_cols:
    if c in policy_features.columns:
        policy_features[c] = policy_features[c].fillna(0)

# years_since_last_claim (for no-claim policies -> big number)
policy_features["years_since_last_claim"] = (
    (policy_features["renewal_date"] - policy_features["last_claim_date"]).dt.days / 365.25
)
policy_features["years_since_last_claim"] = policy_features["years_since_last_claim"].fillna(999)

print("policy_features:", policy_features.shape)
print("Zero-claim policies:", (policy_features["total_claims"]==0).sum(),
      "(", round((policy_features["total_claims"]==0).mean()*100,2), "% )")

display(policy_features.head())


policy_features: (60000, 20)
Zero-claim policies: 7269 ( 12.12 % )


,customer_id,policy_id,policy_start_date,policy_expiry_date,renewal_date,total_claims,total_claim_amount,avg_claim_amount,max_claim_amount,major_claim_count,paid_claims,rejected_claims,pending_claims,claims_last_1_year,claims_last_3_years,last_claim_date,total_claim_amount_last_1_year,largest_claim_amount_last_1_year,had_major_claim_last_1_year,years_since_last_claim
0,C000001,P000001,2023-09-09,2024-09-09,2024-09-09,2.0,513817.0,256908.5,441899.0,0.0,2.0,0.0,0.0,2.0,2.0,2024-07-29,513817.0,441899.0,0.0,0.114990
1,C000002,P000002,2023-02-14,2024-02-14,2024-02-14,2.0,1826187.0,913093.5,1322535.0,1.0,2.0,0.0,0.0,2.0,2.0,2023-10-12,1826187.0,1322535.0,1.0,0.342231
2,C000003,P000003,2023-09-12,2024-09-12,2024-09-12,2.0,4155749.0,2077874.5,2118271.0,2.0,1.0,1.0,0.0,2.0,2.0,2023-12-05,4155749.0,2118271.0,1.0,0.772074
3,C000004,P000004,2024-10-09,2025-10-09,2025-10-09,2.0,196571.0,98285.5,118691.0,0.0,2.0,0.0,0.0,2.0,2.0,2025-05-23,196571.0,118691.0,0.0,0.380561
4,C000005,P000005,2024-02-29,2025-02-28,2025-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaT,0.0,0.0,0.0,999.000000


In [98]:
# Sanity: NCB policies must exist
zero_claim_count = (policy_features["total_claims"] == 0).sum()
total_policies = len(policy_features)

print("Total policies:", total_policies)
print("Zero-claim policies (NCB):", zero_claim_count)
print("NCB %:", round(100 * zero_claim_count / total_policies, 2))

Total policies: 60000
Zero-claim policies (NCB): 7269
NCB %: 12.12


Data-driven thresholds

In [99]:
freq_1y_threshold = policy_features["claims_last_1_year"].quantile(0.90)
freq_3y_threshold = policy_features["claims_last_3_years"].quantile(0.90)

severity_base = policy_features.loc[policy_features["max_claim_amount"] > 0, "max_claim_amount"]
severity_threshold = severity_base.quantile(0.90) if len(severity_base) else 0

recent_base = policy_features.loc[
    policy_features["largest_claim_amount_last_1_year"] > 0,
    "largest_claim_amount_last_1_year"
]
small_claim_threshold = recent_base.quantile(0.50) if len(recent_base) else 0

print("freq_1y_threshold (p90):", freq_1y_threshold)
print("freq_3y_threshold (p90):", freq_3y_threshold)
print("severity_threshold (p90 of max_claim_amount>0):", severity_threshold)
print("small_claim_threshold (median of largest_claim_amount_last_1_year>0):", small_claim_threshold)


freq_1y_threshold (p90): 3.0
freq_3y_threshold (p90): 3.0
severity_threshold (p90 of max_claim_amount>0): 3127881.0
small_claim_threshold (median of largest_claim_amount_last_1_year>0): 1173003.5


In [100]:
policy_features["frequent_claimer_flag"] = (policy_features["claims_last_1_year"] >= freq_1y_threshold).astype(int)
policy_features["high_long_term_frequency_flag"] = (policy_features["claims_last_3_years"] >= freq_3y_threshold).astype(int)
policy_features["high_severity_claimer_flag"] = (policy_features["max_claim_amount"] >= severity_threshold).astype(int)

policy_features["small_frequent_claims_flag"] = (
    (policy_features["claims_last_1_year"] >= freq_1y_threshold) &
    (policy_features["largest_claim_amount_last_1_year"] > 0) &
    (policy_features["largest_claim_amount_last_1_year"] <= small_claim_threshold)
).astype(int)

display(policy_features[[
    "claims_last_1_year","claims_last_3_years","max_claim_amount",
    "largest_claim_amount_last_1_year","frequent_claimer_flag",
    "high_long_term_frequency_flag","high_severity_claimer_flag",
    "small_frequent_claims_flag"
]].head())

,claims_last_1_year,claims_last_3_years,max_claim_amount,largest_claim_amount_last_1_year,frequent_claimer_flag,high_long_term_frequency_flag,high_severity_claimer_flag,small_frequent_claims_flag
0,2.0,2.0,441899.0,441899.0,0,0,0,0
1,2.0,2.0,1322535.0,1322535.0,0,0,0,0
2,2.0,2.0,2118271.0,2118271.0,0,0,0,0
3,2.0,2.0,118691.0,118691.0,0,0,0,0
4,0.0,0.0,0.0,0.0,0,0,0,0


Continuous risk_score

In [101]:
# Frequency score: scale claims_last_1_year into 0..1 using p95 cap
freq_cap = policy_features["claims_last_1_year"].quantile(0.95)
freq_cap = max(freq_cap, 1)  # avoid divide by zero

policy_features["freq_score"] = (
    policy_features["claims_last_1_year"].clip(0, freq_cap) / freq_cap
)

# Severity score: use log to reduce extreme skew, normalize by p95
sev_raw = np.log1p(policy_features["max_claim_amount"].clip(lower=0))
sev_cap = sev_raw.quantile(0.95)
sev_cap = max(sev_cap, 1e-6)

policy_features["sev_score"] = (sev_raw.clip(0, sev_cap) / sev_cap)

# Recency score: recovery effect (recent claim => higher risk)
# Use years_since_last_claim (NCB policies have 999 so they become very low risk here)
policy_features["recency_score"] = np.exp(
    -policy_features["years_since_last_claim"].clip(0, 10) / 5
)

display(policy_features[["freq_score","sev_score","recency_score"]].describe())

,freq_score,sev_score,recency_score
count,60000.000000,60000.000000,60000.000000
mean,0.422421,0.782455,0.831472
std,0.288322,0.305051,0.262806
min,0.000000,0.000000,0.135335
25%,0.250000,0.758799,0.862566
50%,0.250000,0.878332,0.924178
75%,0.500000,0.977064,0.967150
max,1.000000,1.000000,0.999453


In [102]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X_w = policy_features[[
    "freq_score",
    "sev_score",
    "recency_score"
]]

y_w = policy_features["total_claim_amount_last_1_year"]

model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

model.fit(X_w, y_w)

weights = pd.Series(
    model.named_steps["ridge"].coef_,
    index=X_w.columns
)

weights


,0
freq_score,8.843657e+05
sev_score,3.017231e+06
recency_score,-2.738357e+06


In [103]:
weights_norm = weights / weights.abs().sum()
weights_norm

,0
freq_score,0.133189
sev_score,0.454405
recency_score,-0.412406


In [104]:
W_FREQ = weights_norm["freq_score"]
W_SEV  = weights_norm["sev_score"]
W_REC  = weights_norm["recency_score"]

BASE_RISK = 0.02   # portfolio baseline (actuarially normal)

raw = BASE_RISK + (
    W_FREQ * policy_features["freq_score"] +
    W_SEV  * policy_features["sev_score"] +
    W_REC  * policy_features["recency_score"]
)

# Behaviour boosts (small)
raw = raw \
    + 0.10 * policy_features["had_major_claim_last_1_year"] \
    + 0.05 * policy_features["small_frequent_claims_flag"]

policy_features["risk_score"] = (raw.clip(0, 1) * 100).round(2)

display(policy_features["risk_score"].describe())


,risk_score
count,60000.000000
mean,14.332599
std,11.224476
min,0.000000
25%,2.990000
50%,15.470000
75%,24.860000
max,37.130000


In [105]:
print("NCB (0-claim) risk_score summary:")
display(policy_features.loc[policy_features["total_claims"] == 0, "risk_score"].describe())

print("Frequent claimer risk_score summary:")
display(policy_features.loc[policy_features["frequent_claimer_flag"] == 1, "risk_score"].describe())

bins = pd.qcut(policy_features["risk_score"], 5, duplicates="drop")
trend = policy_features.groupby(bins)["total_claim_amount"].mean()

print("Risk-score quintile -> mean total_claim_amount (should increase):")
display(trend)

NCB (0-claim) risk_score summary:


,risk_score
count,7269.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


Frequent claimer risk_score summary:


,risk_score
count,14385.000000
mean,25.938692
std,5.532987
min,7.740000
25%,24.710000
50%,27.710000
75%,29.770000
max,37.130000


Risk-score quintile -> mean total_claim_amount (should increase):


/tmp/ipython-input-157829293.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trend = policy_features.groupby(bins)["total_claim_amount"].mean()


,total_claim_amount
risk_score,
"(-0.001, 1.4]",2.594246e+04
"(1.4, 7.51]",1.940257e+05
"(7.51, 21.37]",1.000572e+06
"(21.37, 25.94]",2.605897e+06
"(25.94, 37.13]",4.059879e+06


In [106]:
policy_features["claims_last_1_year"].describe()

,claims_last_1_year
count,60000.000000
mean,1.735233
std,1.271466
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,11.000000


In [107]:
policy_features["claims_last_1_year"].value_counts().sort_index()

,count
claims_last_1_year,
0.0,7288
1.0,24332
2.0,13995
3.0,8481
4.0,3915
5.0,1421
6.0,433
7.0,106
8.0,21


In [108]:
policy_features["claims_last_1_year"].quantile([0.75, 0.80, 0.90, 0.95])

,claims_last_1_year
0.75,2.0
0.80,3.0
0.90,3.0
0.95,4.0


In [109]:
freq_threshold = policy_features["claims_last_1_year"].quantile(0.90)

policy_features["frequent_claimer_flag"] = (
    policy_features["claims_last_1_year"] >= freq_threshold
).astype(int)

In [110]:
policy_features["max_claim_amount"].describe()

,max_claim_amount
count,6.000000e+04
mean,1.191115e+06
std,1.205756e+06
min,0.000000e+00
25%,8.811750e+04
50%,5.297350e+05
75%,2.330742e+06
max,3.499983e+06


In [111]:
policy_features["max_claim_amount"].quantile([0.75, 0.85, 0.90, 0.95])

,max_claim_amount
0.75,2330742.50
0.85,2841039.60
0.90,3074490.10
0.95,3288279.75


In [112]:
OUT_PATH = "/content/drive/MyDrive/Data/Datasets/renewal_risk_score_dataset.csv"
policy_features.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)

Saved: /content/drive/MyDrive/Data/Datasets/renewal_risk_score_dataset.csv


from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X_w = policy_features[[
    "freq_score",
    "sev_score",
    "recency_score"
]]

y_w = policy_features["total_claim_amount_last_1_year"]

model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge(alpha=1.0))
])

model.fit(X_w, y_w)

weights = pd.Series(
    model.named_steps["ridge"].coef_,
    index=X_w.columns
)

weights


weights_norm = weights / weights.abs().sum()
weights_norm